disco  
Copyright (C) 2022-present NAVER Corp.  
Creative Commons Attribution-NonCommercial-ShareAlike 4.0 license  

# Quasi-Rejection Sampling

Another way to approximate our EBM is to use Quasi-Rejection Sampling.

## Expressing Preferences

That part is similar to the previous notebooks, about the Expression of Preferences or Tuning with DPG. Here we're going to define a distributional constraint: we want half of our examples to ~~be~~include amazing.

In [ ]:
import re
from disco.scorers import BooleanScorer
from disco.distributions import LMDistribution
from disco.distributions.single_context_distribution import SingleContextDistribution

is_amazing = lambda s, c: bool(re.search(r"\bamazing\b", s.text))
amazing_scorer = BooleanScorer(is_amazing)
base = LMDistribution()

incipit = "It was a cold and stormy night"

target = base.constrain([amazing_scorer], [1/2],
        n_samples=2**10,
        context_distribution=SingleContextDistribution(incipit))

## Sampling

We then instantiate a proposal model to sample from.

In [ ]:
proposal = LMDistribution()

Note that we can peek inside the EBM to look at the computed coeeficient

In [ ]:
print(target.scorers[1].coefficients)

It's now just a matter of instantiating a sampler.

In [ ]:
from disco.samplers import QuasiRejectionSampler

In [ ]:
sampler = QuasiRejectionSampler(target, proposal, beta=400)

How are we doing?

In [ ]:
from disco.samplers import AccumulationSampler

In [ ]:
samples, log_scores = sampler.sample(sampling_size=2**10, context=incipit)

In [ ]:
sum([is_amazing(s, _) for s in samples]) / len(samples)